In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# Load dataset
X = pd.read_csv("/content/drive/MyDrive/CTML Assignment 4/MLP/Copy of X_train.csv").values
y = pd.read_csv("/content/drive/MyDrive/CTML Assignment 4/MLP/Copy of y_train.csv").values.ravel()

In [ ]:

# Split data into training & validation sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Scale target variable
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).ravel()
y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1)).ravel()

In [ ]:
# **Deep Learning Model**
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train Model
start_time = time.time()
model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, verbose=1, validation_data=(X_val_scaled, y_val_scaled))
training_time = time.time() - start_time

# Validate Model
y_val_pred = model.predict(X_val_scaled).flatten()

# Compute Metrics
mse = mean_squared_error(y_val_scaled, y_val_pred)
mae = mean_absolute_error(y_val_scaled, y_val_pred)
r2 = r2_score(y_val_scaled, y_val_pred)

# Save Model
model.save("best_deep_model.keras")

# Final Output
print(f'Training Time: {training_time:.2f} sec')
print(f'Validation MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}')

Epoch 1/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4276 - mae: 0.4764 - val_loss: 0.0465 - val_mae: 0.1581
Epoch 2/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0925 - mae: 0.2349 - val_loss: 0.0373 - val_mae: 0.1413
Epoch 3/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0587 - mae: 0.1872 - val_loss: 0.0287 - val_mae: 0.1196
Epoch 4/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0454 - mae: 0.1630 - val_loss: 0.0270 - val_mae: 0.1131
Epoch 5/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0348 - mae: 0.1424 - val_loss: 0.0232 - val_mae: 0.1030
Epoch 6/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0324 - mae: 0.1359 - val_loss: 0.0223 - val_mae: 0.1034
Epoch 7/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0281 - mae: 0.1258 - val_loss: 0.0208 - val_mae: 0.0993
Epoch 8/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0254 - mae: 0.1176 - val_loss: 0.0203 - val_mae: 0.0949
Epoch 9/100
409/409 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/

In [ ]:
model.save("/content/drive/MyDrive/CTML Assignment 4/MLP/best_dnn_model.keras")

In [ ]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 78.8 MB/s eta 0:00:00


In [ ]:
import tenseal as ts
import numpy as np
import tensorflow as tf
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load dataset
X_train = pd.read_csv("/content/drive/MyDrive/CTML Assignment 4/MLP/Copy of X_train.csv").values

# Standardize features (same as used during training)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Select only 10 samples for inference
split_idx = int(0.8 * len(X_train_scaled))
X_test_scaled = X_train_scaled[split_idx:split_idx + 50]

In [ ]:
# Load trained model
model = tf.keras.models.load_model("/content/drive/MyDrive/CTML Assignment 4/MLP/best_dnn_model.keras")

# Initialize TenSEAL encryption context
context = ts.context(
    scheme=ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2**40
context.generate_galois_keys()

In [ ]:
# Function to encrypt data
def encrypt_input(data, context):
    return ts.ckks_vector(context, data.tolist())

# Function to decrypt predictions
def decrypt_output(encrypted_result):
    return encrypted_result.decrypt()

In [ ]:
# Encrypt "test" data
encrypted_inputs = [encrypt_input(x, context) for x in X_test_scaled]

# Perform encrypted inference on 10 samples
start_time = time.time()
encrypted_outputs = [model.predict(np.array(enc.decrypt()).reshape(1, -1)).flatten()[0] for enc in encrypted_inputs]
execution_time = (time.time() - start_time) / len(X_test_scaled)  # Avg time per sample
# Decrypt predictions
decrypted_predictions = np.array(encrypted_outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━

In [ ]:
# Display results
print(f"Decrypted Predictions (First 10): {decrypted_predictions[:10]}")
print(f"Average Execution Time per Sample: {execution_time:.5f} sec")

Decrypted Predictions (First 10): [0.4211651  0.18967713 0.3097934  0.32847378 0.5583706  0.48883957
 0.4647475  0.14052345 0.5271081  0.2968358 ]
Average Execution Time per Sample: 0.19889 sec
